In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as keras

In [5]:
import pathlib
dataset_path = "/kaggle/input/sarscov2-ctscan-dataset"
data_dir = pathlib.Path(dataset_path)

In [6]:
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)

In [7]:
covid = list(data_dir.glob('COVID/*'))
PIL.Image.open(str(covid[1]))

In [8]:
ncovid = list(data_dir.glob('non-COVID/*'))
PIL.Image.open(str(ncovid[1]))

In [9]:
batch_size = 32
img_height = 180
img_width = 180

In [10]:
train_ds = keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode="grayscale"
)

In [11]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode="grayscale" 
)

In [12]:
class_names = train_ds.class_names
print(class_names)

In [13]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"),cmap=plt.get_cmap("gray"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [14]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [15]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[180,180]),
    keras.layers.Rescaling(1./255),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(2, activation="softmax")
])

In [16]:
model.summary()

In [17]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [18]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot()
plt.grid(True)
# plt.gca().set_ylim(0, 1)
# save_fig("keras_learning_curves_plot")
plt.show()

In [20]:
num_classes = 2

model7 = keras.Sequential([
  keras.layers.Rescaling(1./255),
  keras.layers.Conv2D(32, 1, activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 1, activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 1, activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes)
])

In [21]:
model7.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [22]:
his=model7.fit(
  train_ds,
  validation_data=val_ds,
  epochs=4
)

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(his.history).plot(figsize=(10, 5))
plt.grid(True)
# plt.gca().set_ylim(0, 1)
# save_fig("keras_learning_curves_plot")
plt.show()